<a href="https://colab.research.google.com/github/OmerTurk1/ML-From-Scratch/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
import numpy as np

In [ ]:
def create_dataset(n=300):
    vals1 = np.random.randint(1, 151, size=n)
    vals2 = np.random.randint(1, 151, size=n)
    df = pd.DataFrame({
        "val1": vals1,
        "val2": vals2,
        "cls": ((vals1 <= 50) | ((vals1 <= 100) & (vals2 <= 50)) | (vals2 >= 100)).astype(int)
    })
    return df

In [ ]:
df = create_dataset()
df.head()

,val1,val2,cls
0,94,81,0
1,144,149,1
2,147,79,0
3,62,113,1
4,89,52,0


In [ ]:
class Node:
    def __init__(self, tree, value=None, depth=0, cond=None):
        self.right = None
        self.left = None
        self.value = value
        self.g = 0
        self.cond = cond
        self.depth = depth
        self.tree = tree

    def gini(self, arr):
        counts = arr.value_counts(normalize=True)
        return 1 - (counts ** 2).sum()

    def predict(self, samp):
        if self.right==None and self.left==None:
            return self.value # o node'un değeri (en çok bulunan öge)
        elif self.cond(samp):
            return self.left.predict(samp)
        else:
            return self.right.predict(samp)

    def best_split(self,mini_df,target,var):
        mini_df = mini_df.sort_values(by=var)
        result = [-1,2] # sınır, gini, sınırın altı, sınırın üstü
        st = sorted(list(set(mini_df[var])))
        for i in range(len(st)-1):
            s = (st[i]+st[i+1])/2
            lower_targ = mini_df[mini_df[var]<s][target]
            upper_targ = mini_df[mini_df[var]>=s][target]
            weighted = (len(lower_targ)*self.gini(lower_targ) + len(upper_targ)*self.gini(upper_targ))/(mini_df.shape[0])
            if result[1]>weighted:
                result=[s,weighted, lower_targ.mode()[0], upper_targ.mode()[0]]
        return result

    def best_choice(self,X,Y):
        result=[-1,2,""] # sırasıyla sınır, gini ve sütun ismi
        for x in X.columns:
            mini_df=pd.DataFrame({x: X[x],Y.name: Y})
            candi = self.best_split(mini_df=mini_df,target=Y.name,var=x) + [x]
            if result[1]>candi[1]:
                result=candi
        return result

    def fit(self, X, Y):
        self.value = Y.mode()[0]
        if (
            Y.value_counts(normalize=True).max() >= 0.95
            or Y.value_counts().sum() <= 10
            or self.depth == self.tree.max_depth
        ):
            return True
        # en mantıklı splitin yapılabileceği sütunu bul ve spliti döndür
        result = self.best_choice(X,Y)
        self.g = result[1]
        self.cond = lambda samp: True if samp[result[4]]<result[0] else False

        self.right = Node(tree=tree, depth=self.depth+1)
        self.left = Node(tree=tree, depth=self.depth+1)

        mask = X.apply(self.cond, axis=1)
        self.left.fit(X[mask], Y[mask])
        self.right.fit(X[~mask], Y[~mask])

In [ ]:
class Tree:
    def __init__(self, max_depth):
        self.root = Node(depth=0, tree=self)
        self.max_depth = max_depth

    def predict(self, df):
        return self.root.predict(df)

    def fit(self, X, Y):
        self.root.fit(X, Y)

In [ ]:
X = df.drop(columns="cls")
Y=df["cls"]

In [ ]:
%%time
tree = Tree(max_depth=3)
tree.fit(X, Y)

CPU times: user 830 ms, sys: 1.69 ms, total: 832 ms
Wall time: 839 ms


In [ ]:
samp = pd.Series({"val1":97,"val2":47})
tree.predict(samp)

np.int64(0)